In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelWithLMHead, AutoModel
from transformers import Trainer, TrainingArguments
import transformers
transformers.set_seed(42)
import dotenv
import wandb
import os

dotenv.load_dotenv("./.env", override=True)
wandb.login(key=os.getenv('WANDB_API_KEY'))

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find create_dataset.ipynb.
wandb: Currently logged in as: felix-ml (fml-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/felix/.netrc


True

In [2]:

# Specify the model checkpoint
model_checkpoint = "dbmdz/german-gpt2"
# load the model
model = AutoModel.from_pretrained(model_checkpoint)
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
# import langchain json dataset loader
from langchain.document_loaders import JSONLoader
from IPython.display import display, Markdown

import json
from pathlib import Path
from pprint import pprint


file_path = 'data/berufslexikon_regex_cleaned.json'

data = json.loads(Path(file_path).read_text())


In [4]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["profession"] = record.get("profession")
    metadata["url"] = record.get("url")

    return metadata

jq_schema='.[]'
loader = JSONLoader(
    file_path=file_path,
    jq_schema=jq_schema,
    content_key="content",
    metadata_func=metadata_func
)
data = loader.load()

In [5]:
display(Markdown(f"Loaded {len(data)} documents."))

Loaded 1403 documents.

In [6]:
data[0]

Document(page_content='24-Stunden-BetreuerIn\nZurück\n24-Stunden-BetreuerIn\nBerufsbereiche:\nSoziales, Gesundheit, Schönheitspflege\nAusbildungsform: BMS/BHS\nEinstiegsgehalt lt. KV:\nGehalt:\nBMS:\n€ 2.050,- bis € 2.230,- *\n24-Stunden-BetreuerInnen unterstützen pflegebedürftige Menschen, die eine permanente Betreuung benötigen, bei diesen zu Hause. Sie leisten Hilfe bei der täglichen Körperpflege und Hygiene, z.B. beim An- und Auskleiden, Waschen und Duschen, sowie bei der Nahrungsaufnahme und Medikamenteneinnahme. Ein wichtiger Teil sind soziale Aktivitäten und die aktivierende Betreuung: Gesellschaft leisten, die betreute Person strukturiert durch den Tag führen, mit ihr Spaziergänge und kurze Ausflüge machen, auf Wunsch Geschichten vorlesen und Ähnliches.\nWeiters sind 24-Stunden-BetreuerInnen für die Hauswirtschaft und Einkäufe zuständig. Sie stellen die Versorgung mit Lebensmitteln sicher, räumen auf, waschen das Geschirr ab, kümmern sich um die Wäsche und bügeln. Auch leichte 

Chunk

In [7]:
# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        #disallowed_special=()
    )
    return len(tokens)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [9]:
from uuid import uuid4
from tqdm.auto import tqdm
from pathlib import Path
import json

# Define the path
path = Path('data/chunks.json')

# Check if the file exists
if not path.exists():
    chunks = []

    for idx, page in enumerate(tqdm(data)):
        content = page.page_content
        if len(content) > 5:
            texts = text_splitter.split_text(content)
            profession = page.metadata["profession"]
            total_chunks = len(texts)
            chunks.extend([{
                'id': f"{profession}_{i+1}_{total_chunks}",
                'profession': profession,
                'text': texts[i],
                'chunk': i,
                'url': page.metadata["url"]
            } for i in range(total_chunks)])
    # Save the chunks
    path.write_text(json.dumps(chunks, indent=4))
else:
    print("File already exists. Skipping write.")
    # Load the chunks
    chunks = json.loads(path.read_text())

100%|██████████| 1403/1403 [00:13<00:00, 107.00it/s]


In [10]:
chunks[0]

{'id': '24-Stunden-BetreuerIn_1_4',
 'profession': '24-Stunden-BetreuerIn',
 'text': '24-Stunden-BetreuerIn\nZurück\n24-Stunden-BetreuerIn\nBerufsbereiche:\nSoziales, Gesundheit, Schönheitspflege\nAusbildungsform: BMS/BHS\nEinstiegsgehalt lt. KV:\nGehalt:\nBMS:\n€ 2.050,- bis € 2.230,- *\n24-Stunden-BetreuerInnen unterstützen pflegebedürftige Menschen, die eine permanente Betreuung benötigen, bei diesen zu Hause. Sie leisten Hilfe bei der täglichen Körperpflege und Hygiene, z.B. beim An- und Auskleiden, Waschen und Duschen, sowie bei der Nahrungsaufnahme und Medikamenteneinnahme. Ein wichtiger Teil sind soziale Aktivitäten und die aktivierende Betreuung: Gesellschaft leisten, die betreute Person strukturiert durch den Tag führen, mit ihr Spaziergänge und kurze Ausflüge machen, auf Wunsch Geschichten vorlesen und Ähnliches.\nWeiters sind 24-Stunden-BetreuerInnen für die Hauswirtschaft und Einkäufe zuständig. Sie stellen die Versorgung mit Lebensmitteln sicher, räumen auf, waschen das Ge

# Chroma

In [11]:
import chromadb.utils.embedding_functions as embedding_functions

# hugginface api key
hf_api_key = os.getenv('HUGGINGFACE_API_KEY_READ')
embedding_function_hf = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=hf_api_key,
    model_name="sentence-transformers/all-MiniLM-L6-v2", # do not use LLM!!!
)

In [12]:
# if you're having an sqlite3 error, you may have to uncomment this line see: https://docs.trychroma.com/troubleshooting#sqlite
#import settings 
import chromadb

chroma_client = chromadb.PersistentClient() # Equivalent to chromadb.Client(), persistent.
#chroma_client = chromadb.EphemeralClient()

ams_content_collection = chroma_client.get_or_create_collection(name='ams_content', embedding_function=embedding_function_hf)


In [13]:
# check for cuda
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('Device name:', torch.cuda.get_device_name(0))
    

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3090


In [21]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x.get('id') for x in meta_batch]
    # get texts to encode
    texts = [x.get('text') for x in meta_batch]
    
    # create dict for metadata
    # get urls and professions
    metadata = [{'profession': x.get('profession'), 'url': x.get('url')} for x in meta_batch]
    
    ams_content_collection.add(
        documents=texts, #Mit Chroma kann ich die Daten direkt adden
        metadatas=metadata,
        ids=ids_batch
    )

  0%|          | 0/62 [00:00<?, ?it/s]Add of existing embedding ID: 24-Stunden-BetreuerIn_1_4
Add of existing embedding ID: 24-Stunden-BetreuerIn_2_4
Add of existing embedding ID: 24-Stunden-BetreuerIn_3_4
Add of existing embedding ID: 24-Stunden-BetreuerIn_4_4
Add of existing embedding ID: 3D-DesignerIn_1_4
Add of existing embedding ID: 3D-DesignerIn_2_4
Add of existing embedding ID: 3D-DesignerIn_3_4
Add of existing embedding ID: 3D-DesignerIn_4_4
Add of existing embedding ID: AbfallbeauftragteR_1_2
Add of existing embedding ID: AbfallbeauftragteR_2_2
Add of existing embedding ID: AbfallberaterIn_1_3
Add of existing embedding ID: AbfallberaterIn_2_3
Add of existing embedding ID: AbfallberaterIn_3_3
Add of existing embedding ID: AbfallwirtschaftstechnikerIn_1_4
Add of existing embedding ID: AbfallwirtschaftstechnikerIn_2_4
Add of existing embedding ID: AbfallwirtschaftstechnikerIn_3_4
Add of existing embedding ID: AbfallwirtschaftstechnikerIn_4_4
Add of existing embedding ID: Abnahmet

In [23]:
ams_content_collection.query(query_texts="tiere", n_results=10, include=['documents']) 

{'ids': [['TierhändlerIn_1_1',
   'TiertrainerIn_2_2',
   'TiertrainerIn_1_2',
   'Tierarzt/-ärztin für Tiermedizinische Forschung_1_2',
   'HundetrainerIn_3_3',
   'TierpflegerIn_2_8',
   'TierpflegerIn_5_8',
   'Tierarzt/-ärztin_1_3',
   'Tierarzt/-ärztin für Veterinärverwaltung_1_2',
   'TierpflegerIn_1_8']],
 'distances': None,
 'metadatas': None,
 'embeddings': None,
 'documents': [['TierhändlerIn\nZurück\nTierhändlerIn\nBerufsbereiche:\nHandel, Logistik, Verkehr\n/ Landwirtschaft, Gartenbau, Forstwirtschaft\nAusbildungsform: Hilfs-/Anlernberufe\nEinstiegsgehalt lt. KV:\nGehalt:\n€ 1.810,- bis € 2.190,- *\nTierhändlerInnen kaufen, züchten und verkaufen Haustiere, wie z.B. Hunde, Katzen, Meerschweinchen, Wellensittiche\xa0oder Zierfische. Sie handeln auch mit Tierbedarfsartikeln. Beispiele dafür sind Leinen, Maulkörbe, Bürsten und Futtermittel. TierhändlerInnen bestellen nachgefragte Tiere oder züchten sie selbst und berechnen die Preise. Sie gestalten die Geschäftsflächen im Zooge